In [1]:
import os
import pandas as pd
import numpy as np
#import matplotlib
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter
import matplotlib.ticker as ticker
import matplotlib.dates as mdates
%matplotlib inline
from ebmdatalab import bq
from ebmdatalab import charts

**PRICE CONCESSION DATA**

In [2]:
#get price concession data from BigQuery
sql = """
  SELECT DISTINCT
    ncso.vmpp AS vmpp,
    ncso.date AS month,    
    1 AS concession_bool --creates a boolean value to show a price concession exists
  FROM
    ebmdatalab.dmd.ncsoconcession AS ncso --concession table 
"""
exportfile = os.path.join("..","data","ncso_dates.csv") #defines name for cache file
dates_df = bq.cached_read(sql, csv_path=exportfile, use_cache=False) #uses BQ if changed, otherwise csv cache file
dates_df['month'] = pd.to_datetime(dates_df['month']) #ensure dates are in datetimeformat
dates_df = dates_df.sort_values(by=['month','vmpp']) #sort data by month then vmpp
dates_df.head()

Downloading: 100%|██████████| 6578/6578 [00:00<00:00, 12604.35rows/s]


,vmpp,month,concession_bool
2700,959311000001107,2014-08-01,1
2991,975311000001108,2014-08-01,1
3930,1034211000001109,2014-08-01,1
3998,1045211000001109,2014-08-01,1
920,1122311000001100,2014-08-01,1


In [3]:
#unstacks data, fills missing month data (with zero value where no concession), then restacks
dates_cons_df = dates_df.set_index(['month','vmpp']).unstack().asfreq('MS').fillna(0).stack().sort_index(level=1).reset_index()
#dates_cons_df = dates_cons_df.loc[dates_cons_df['vmpp'] == 1040511000001102]
dates_cons_df.head()

,month,vmpp,concession_bool
0,2014-08-01,941211000001102,0.0
1,2014-09-01,941211000001102,0.0
2,2014-10-01,941211000001102,0.0
3,2014-11-01,941211000001102,0.0
4,2014-12-01,941211000001102,0.0


In [4]:
max_date = dates_cons_df["month"].max() + pd.DateOffset(months=-3) #creates variable to ensure that all price concession data have three months after concession ends to ensure calculation of change
pc_summary_df = (dates_cons_df.assign(Consecutive=dates_cons_df.concession_bool
                                .groupby((dates_cons_df.concession_bool != dates_cons_df.concession_bool.shift())
                                    .cumsum()).transform('size')) #creates a value of the number of consecutive months of either price concession or no price concession
          .query('concession_bool > 0') # filters to only where price concession is present 
          .groupby(['vmpp','Consecutive'])
          .aggregate(first_month=('month','first'),  #shows earliest month of consecutive price concession
                     last_month=('month','last')) #shows latest month of consecutive price concession
          .reset_index().query("last_month < @max_date")
          .reset_index(drop=True)
)

pc_summary_df.head()

,vmpp,Consecutive,first_month,last_month
0,941211000001102,3,2015-11-01,2016-01-01
1,941211000001102,5,2019-11-01,2020-03-01
2,941311000001105,2,2017-11-01,2017-12-01
3,941311000001105,8,2022-05-01,2022-12-01
4,941511000001104,6,2022-04-01,2022-09-01


In [5]:
#get drug tariff price data from BigQuery
sql = """
  SELECT 
    vmpp.bnf_code as bnf_code, --BNF code (at VMP level)
    vmpp.nm as nm, --name
    vmpp.qtyval as unit_qty, --quantity per pack
    dt.*
  FROM
    ebmdatalab.dmd.tariffprice AS dt --concession table
    INNER JOIN
    dmd.vmpp as vmpp --join to VMPP table to get BNF codes and names
    on
    dt.vmpp = vmpp.id
  WHERE
    dt.vmpp IN (SELECT DISTINCT vmpp FROM ebmdatalab.dmd.ncsoconcession)
"""

exportfile = os.path.join("..","data","tariff.csv") #defines name for cache file
dates_df = bq.cached_read(sql, csv_path=exportfile, use_cache=False) #uses BQ if changed, otherwise csv cache file
dates_df['date'] = pd.to_datetime(dates_df['date'])#ensure dates are in datetimeformat
dates_df['unit_qty'] = pd.to_numeric(dates_df['unit_qty'])

Downloading: 100%|██████████| 102327/102327 [00:07<00:00, 14211.61rows/s]


In [6]:
dates_df['pre_month'] = dates_df['date'] + pd.DateOffset(months=1) #creates extra date column in drug tariff price shifted by one month later, to pick up 3 month rolling mean spend for the month before price concession added
dates_df['post_month'] = dates_df['date'] + pd.DateOffset(months=-3) #creates extra date column in drug tariff price shifted by three months earlier, to pick up 3 month rolling mean spend for the 3 months after price concession added
dates_df['3_month_price'] = dates_df.groupby('vmpp')['price_pence'].transform(lambda x: x.rolling(3, 3).mean()) # create three month rolling average drug tariff cost

In [7]:
dates_df_merge = pd.merge(pc_summary_df, dates_df[['bnf_code', 'nm','unit_qty','vmpp','pre_month','3_month_price']],  how='left', left_on=['vmpp','first_month'], right_on = ['vmpp','pre_month']) #merges price concession information with the 3 month average DT price prior to the start of the price concession
dates_df_merge.rename(columns={'3_month_price' : 'pre_pc_price'}, inplace=True) #rename columns
dates_df_merge = pd.merge(dates_df_merge, dates_df[['vmpp','post_month','3_month_price']],  how='left', left_on=['vmpp','last_month'], right_on = ['vmpp','post_month']) #merges price concession information with the 3 month average DT price after the end of the price concession
dates_df_merge.rename(columns={'3_month_price' : 'post_pc_price'}, inplace=True) #rename columns
dates_df_merge = dates_df_merge.drop(columns=['pre_month', 'post_month']) #drop unneccesary columns
dates_df_merge = dates_df_merge.sort_values(by=['vmpp','first_month']) #sort data by month then vmpp
dates_df_merge['perc_difference'] = (dates_df_merge['post_pc_price']/dates_df_merge['pre_pc_price']-1)
dates_df_merge['rx_merge_date'] = (dates_df_merge['last_month'] + pd.DateOffset(months=1)) #create a merge date for prescribing data, so there's always the three months of rx data available post concession
dates_df_merge = dates_df_merge.sort_values(by=['last_month'], ascending=False) #sort data by month then vmpp

In [8]:
dates_df_merge.head()

,vmpp,Consecutive,first_month,last_month,bnf_code,nm,unit_qty,pre_pc_price,post_pc_price,perc_difference,rx_merge_date
349,1128011000001100,2,2023-04-01,2023-05-01,0403040W0AAADAD,Venlafaxine 75mg modified-release capsules 28 ...,28.0,286.000000,329.000000,0.150350,2023-06-01
141,1006611000001106,6,2022-12-01,2023-05-01,0501013B0AAAJAJ,Amoxicillin 125mg/5ml oral suspension 100 ml,100.0,127.000000,323.000000,1.543307,2023-06-01
198,1034411000001108,9,2015-01-01,2023-05-01,0201010F0AAADAD,Digoxin 62.5microgram tablets 28 tablet,28.0,148.000000,411.333333,1.779279,2023-06-01
948,19182011000001104,8,2022-10-01,2023-05-01,0906040G0AABHBH,"Colecalciferol 1,000unit capsules 30 capsule",30.0,1000.000000,562.333333,-0.437667,2023-06-01
720,3410511000001107,1,2023-05-01,2023-05-01,0703021Q0AAAAAA,Desogestrel 75microgram tablets 84 tablet,84.0,261.666667,331.000000,0.264968,2023-06-01


In [9]:
#get quantity_calcs
sql = """
  SELECT DISTINCT
    date(rx.month) as date_3m_start,
    rx.bnf_code,
    SUM(rx.quantity) OVER(
      PARTITION BY rx.bnf_code
      ORDER BY DATE_DIFF(date(rx.month), '2000-01-01', MONTH)
      RANGE BETWEEN 0 PRECEDING AND 2 FOLLOWING
    )
    as roll_3m_quantity
  FROM
    ebmdatalab.hscic.normalised_prescribing AS rx
    INNER JOIN
    dmd.vmpp as vmpp --join to VMPP table to get BNF codes and names
    on
    rx.bnf_code = vmpp.bnf_code
  WHERE
    vmpp.id IN (SELECT DISTINCT vmpp FROM ebmdatalab.dmd.ncsoconcession)
    AND rx.pct = '15N'
    AND month >='2022-04-01'
    ORDER BY date_3m_start DESC
"""

exportfile = os.path.join("..","data","rx_qty.csv") #defines name for cache file
rx_df = bq.cached_read(sql, csv_path=exportfile, use_cache=True) #uses BQ if changed, otherwise csv cache file
rx_df['date_3m_start'] = pd.to_datetime(rx_df['date_3m_start'])#ensure dates are in datetimeformat
rx_df = rx_df[rx_df['date_3m_start'] <= max(rx_df['date_3m_start']) + pd.DateOffset(months=-2)] #limit df to ensure that always 3 full months of data


In [10]:
rx_df.head()

,date_3m_start,bnf_code,roll_3m_quantity
1265,2023-04-01,0408010AEAAABAB,437171.0
1266,2023-04-01,0206020K0AAABAB,20482.0
1267,2023-04-01,0205040D0AAABAB,363275.0
1268,2023-04-01,0501110C0AAAJAJ,46283.0
1269,2023-04-01,1106000L0AAAAAA,70090.0


In [11]:
rx_df_merge = pd.merge(dates_df_merge, rx_df,  how='right', left_on=['bnf_code','rx_merge_date'], right_on = ['bnf_code','date_3m_start'])

In [12]:
rx_df_merge.head()

,vmpp,Consecutive,first_month,last_month,bnf_code,nm,unit_qty,pre_pc_price,post_pc_price,perc_difference,rx_merge_date,date_3m_start,roll_3m_quantity
0,1.261911e+15,1.0,2023-03-01,2023-03-01,0501021L0AAABAB,Cefalexin 500mg capsules 21 capsule,21.0,248.000000,245.0,-0.012097,2023-04-01,2023-04-01,57491.0
1,7.890711e+15,4.0,2022-12-01,2023-03-01,0408010AEAAADAD,Pregabalin 100mg capsules 84 capsule,84.0,280.333333,245.0,-0.126040,2023-04-01,2023-04-01,371931.0
2,9.760110e+14,9.0,2022-07-01,2023-03-01,0702020F0AAAFAF,Clotrimazole 500mg pessaries 1 pessary,1.0,463.000000,979.0,1.114471,2023-04-01,2023-04-01,1760.0
3,1.212011e+15,7.0,2022-09-01,2023-03-01,1001010P0AAAIAI,Naproxen 500mg gastro-resistant tablets 56 tablet,56.0,446.666667,567.0,0.269403,2023-04-01,2023-04-01,71659.0
4,9.423110e+14,4.0,2022-12-01,2023-03-01,1103010H0AAAAAA,Fusidic acid 1% modified-release eye drops 5 gram,5.0,2405.666667,3428.0,0.424969,2023-04-01,2023-04-01,1195.0


In [13]:
rx_df_merge['3_m_additional_cost'] = 0.01*(rx_df_merge['roll_3m_quantity']/rx_df_merge['unit_qty'])*(rx_df_merge['post_pc_price']-rx_df_merge['pre_pc_price'])

In [14]:
rx_df_merge.head()

,vmpp,Consecutive,first_month,last_month,bnf_code,nm,unit_qty,pre_pc_price,post_pc_price,perc_difference,rx_merge_date,date_3m_start,roll_3m_quantity,3_m_additional_cost
0,1.261911e+15,1.0,2023-03-01,2023-03-01,0501021L0AAABAB,Cefalexin 500mg capsules 21 capsule,21.0,248.000000,245.0,-0.012097,2023-04-01,2023-04-01,57491.0,-82.130000
1,7.890711e+15,4.0,2022-12-01,2023-03-01,0408010AEAAADAD,Pregabalin 100mg capsules 84 capsule,84.0,280.333333,245.0,-0.126040,2023-04-01,2023-04-01,371931.0,-1564.471667
2,9.760110e+14,9.0,2022-07-01,2023-03-01,0702020F0AAAFAF,Clotrimazole 500mg pessaries 1 pessary,1.0,463.000000,979.0,1.114471,2023-04-01,2023-04-01,1760.0,9081.600000
3,1.212011e+15,7.0,2022-09-01,2023-03-01,1001010P0AAAIAI,Naproxen 500mg gastro-resistant tablets 56 tablet,56.0,446.666667,567.0,0.269403,2023-04-01,2023-04-01,71659.0,1539.815417
4,9.423110e+14,4.0,2022-12-01,2023-03-01,1103010H0AAAAAA,Fusidic acid 1% modified-release eye drops 5 gram,5.0,2405.666667,3428.0,0.424969,2023-04-01,2023-04-01,1195.0,2443.376667


In [15]:
exportfile = os.path.join("..","data","3_months_post.csv") #defines name for cache file
rx_df_merge.to_csv(exportfile, index=False)  

In [44]:
rx_sum_df = rx_df_merge.groupby('date_3m_start')['3_m_additional_cost'].sum().to_frame()

In [45]:
rx_sum_df.head(200)

,3_m_additional_cost
date_3m_start,
2022-04-01,-8078.808646
2022-05-01,22352.376092
2022-06-01,3759.962476
2022-07-01,290412.515198
2022-08-01,26404.077262
2022-09-01,9319.957722
2022-10-01,491653.845558
2022-11-01,20281.815203
2022-12-01,2053.364663


In [47]:
table = rx_df_merge.pivot_table(rx_df_merge, values=['3_m_additional_cost'], index=['date_3m_start'], columns=['nm'], fill_value=0, aggfunc=np.sum, dropna=True)

TypeError: pivot_table() got multiple values for argument 'values'